In [24]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 200)
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

import time

from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import GridSearchCV, KFold

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge, ElasticNet

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

## Defining files names
file_key_1 = 'Store-Sales-Time-Series-Forecast/oil.csv'
file_key_2 = 'Store-Sales-Time-Series-Forecast/holidays_events.csv'
file_key_3 = 'Store-Sales-Time-Series-Forecast/stores.csv'
file_key_4 = 'Store-Sales-Time-Series-Forecast/transactions.csv'
file_key_5 = 'Store-Sales-Time-Series-Forecast/train.csv'
file_key_6 = 'Store-Sales-Time-Series-Forecast/test.csv'
file_key_7 = 'Store-Sales-Time-Series-Forecast/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

bucket_object_5 = bucket.Object(file_key_5)
file_object_5 = bucket_object_5.get()
file_content_stream_5 = file_object_5.get('Body')

bucket_object_6 = bucket.Object(file_key_6)
file_object_6 = bucket_object_6.get()
file_content_stream_6 = file_object_6.get('Body')

bucket_object_7 = bucket.Object(file_key_7)
file_object_7 = bucket_object_7.get()
file_content_stream_7 = file_object_7.get('Body')

## Reading data-files
oil = pd.read_csv(file_content_stream_1)
holidays = pd.read_csv(file_content_stream_2)
stores = pd.read_csv(file_content_stream_3)
transactions = pd.read_csv(file_content_stream_4)
train = pd.read_csv(file_content_stream_5)
test = pd.read_csv(file_content_stream_6)
submission = pd.read_csv(file_content_stream_7)

# Train dataset

In [25]:
holidays.columns = ['date', 'holiday_type', 'locale', 'locale_name', 'description', 'transferred']
stores.columns = ['store_nbr', 'city', 'state', 'store_type', 'cluster']

train = pd.merge(train, oil, on = 'date', how = 'left')
train = pd.merge(train, holidays, on = 'date', how = 'left')
train = pd.merge(train, stores, on = 'store_nbr', how = 'left')
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

## Basic feature engineering 
cluster_dummies = pd.get_dummies(train['cluster'])
cluster_dummies.columns = ['cluster_' + str(i) for i in range(1, 18)]
train = pd.concat([train.drop(columns = ['cluster'], axis = 1), cluster_dummies], axis = 1)

family_dummies = pd.get_dummies(train['family'])
family_dummies.columns = ['family_' + str(i) for i in range(1, 34)]
train = pd.concat([train.drop(columns = ['family'], axis = 1), family_dummies], axis = 1)

train['day'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['is_holiday'] = np.where(train['holiday_type'] == 'Holiday', 1, 0)

train = train.fillna(0)

train.head()

,id,date,store_nbr,sales,onpromotion,dcoilwtico,holiday_type,locale,locale_name,description,transferred,city,state,store_type,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17,family_1,family_2,family_3,family_4,family_5,family_6,family_7,family_8,family_9,family_10,family_11,family_12,family_13,family_14,family_15,family_16,family_17,family_18,family_19,family_20,family_21,family_22,family_23,family_24,family_25,family_26,family_27,family_28,family_29,family_30,family_31,family_32,family_33,day,month,is_holiday
0,0,2013-01-01,1,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
1,1,2013-01-01,1,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
2,2,2013-01-01,1,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
3,3,2013-01-01,1,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
4,4,2013-01-01,1,0.0,0,0.0,Holiday,National,Ecuador,Primer dia del ano,False,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1


# Test dataset

In [26]:
## Appending oil prices and holiday
test = pd.merge(test, holidays, on = 'date', how = 'left')
test = pd.merge(test, oil, on = 'date', how = 'left')
test = pd.merge(test, stores, on = 'store_nbr', how = 'left')
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

## Basic feauture engineering 
cluster_dummies = pd.get_dummies(test['cluster'])
cluster_dummies.columns = ['cluster_' + str(i) for i in range(1, 18)]
test = pd.concat([test.drop(columns = ['cluster'], axis = 1), cluster_dummies], axis = 1)

family_dummies = pd.get_dummies(test['family'])
family_dummies.columns = ['family_' + str(i) for i in range(1, 34)]
test = pd.concat([test.drop(columns = ['family'], axis = 1), family_dummies], axis = 1)

test['day'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['is_holiday'] = np.where(test['holiday_type'] == 'Holiday', 1, 0)

test = test.fillna(0)

test.head()

,id,date,store_nbr,onpromotion,holiday_type,locale,locale_name,description,transferred,dcoilwtico,city,state,store_type,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17,family_1,family_2,family_3,family_4,family_5,family_6,family_7,family_8,family_9,family_10,family_11,family_12,family_13,family_14,family_15,family_16,family_17,family_18,family_19,family_20,family_21,family_22,family_23,family_24,family_25,family_26,family_27,family_28,family_29,family_30,family_31,family_32,family_33,day,month,is_holiday
0,3000888,2017-08-16,1,0,0,0,0,0,0,46.8,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,0
1,3000889,2017-08-16,1,0,0,0,0,0,0,46.8,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,0
2,3000890,2017-08-16,1,2,0,0,0,0,0,46.8,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,0
3,3000891,2017-08-16,1,20,0,0,0,0,0,46.8,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,0
4,3000892,2017-08-16,1,0,0,0,0,0,0,46.8,Quito,Pichincha,D,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,0


# Linear Regression Models

In [27]:
X = train.drop(columns = ['id', 'date', 'store_nbr', 'sales', 'holiday_type', 'locale', 'locale_name', 'description', 'transferred', 'city', 'state', 'store_type'], axis = 1)
Y = train['sales']

test = test.drop(columns = ['id', 'date', 'store_nbr', 'holiday_type', 'locale', 'locale_name', 'description', 'transferred', 'city', 'state', 'store_type'], axis = 1)

scaler = MinMaxScaler()

t1 = time.time()
kf = KFold(n_splits = 5, shuffle = True, random_state = 888)
score_list_lgb = []
test_preds_lgb = []
fold = 1

for train_index, test_index in kf.split(X, Y):
    
    ## Splitting the data
    X_train , X_val = X.iloc[train_index], X.iloc[test_index]  
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[test_index]    
    
    print('X_train shape is :', X_train.shape, 'X_val shape is', X_val.shape)
    y_pred_list = []
    
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_val = pd.DataFrame(scaler.fit_transform(X_val), columns = X_val.columns)
    
    model_lgb = Lasso(alpha = 0.01, tol = 1e-2, max_iter = 1000000) 
                      
    model = model_lgb.fit(X_train, Y_train)
    result = model_lgb.predict(X_val)
    
    result = pd.DataFrame(result)
    result.iloc[:, 0] = [0 if i <= 0 else i for i in result.iloc[:,0]]
    
    score = np.sqrt(mean_squared_log_error(Y_val, result))
    print('Fold ', str(fold), ' result is:', score, '\n')
    score_list_lgb.append(score)

    test_preds_lgb.append(model_lgb.predict(test))
    fold +=1

t2 = time.time()
print("Lasso model with cross validation take : {:.3f} sn.".format(t2-t1))

X_train shape is : (2443478, 55) X_val shape is (610870, 55)
Fold  1  result is: 2.478104566324058 

X_train shape is : (2443478, 55) X_val shape is (610870, 55)
Fold  2  result is: 2.477494895378721 

X_train shape is : (2443478, 55) X_val shape is (610870, 55)
Fold  3  result is: 2.4760344460612287 

X_train shape is : (2443479, 55) X_val shape is (610869, 55)
Fold  4  result is: 2.473487251910008 

X_train shape is : (2443479, 55) X_val shape is (610869, 55)
Fold  5  result is: 2.4807554644645076 

Lasso model with cross validation take : 598.960 sn.
